# hvPlot Widgets

Your parents need help identifying the best states to move to that have the lowest Diabetes care costs. Use hvPlot and its widgets to visualize hospital claims data in order to identify potential states for recommendation.

In [41]:
import pandas as pd
import hvplot.pandas
from pathlib import Path

### Step 1: Prep Data

In [42]:
# Read in hospital claims data
hospital_data = pd.read_csv(
    Path("../Resources/hospital_claims.csv"), 
    header=0
)

# View the DataFame
hospital_data.head()

,DRG Definition,Provider Id,Provider Name,Provider Street Address,Provider City,Provider State,Provider Zip Code,Hospital Referral Region Description,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,AL - Dothan,91,32963.07,5777.24,4763.73
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,AL - Birmingham,14,15131.85,5787.57,4976.71
2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,AL - Birmingham,24,37560.37,5434.95,4453.79
3,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10011,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,AL - Birmingham,25,13998.28,5417.56,4129.16
4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10016,SHELBY BAPTIST MEDICAL CENTER,1000 FIRST STREET NORTH,ALABASTER,AL,35007,AL - Birmingham,18,31633.27,5658.33,4851.44


In [43]:
# Slice the data for the DRG Definition "638 - DIABETES W CC"
procedure_638_charges = hospital_data[
    hospital_data["DRG Definition"] == "638 - DIABETES W CC"
]

# View the DataFrame
procedure_638_charges.head()

,DRG Definition,Provider Id,Provider Name,Provider Street Address,Provider City,Provider State,Provider Zip Code,Hospital Referral Region Description,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
125166,638 - DIABETES W CC,330004,KINGSTON HOSPITAL,396 BROADWAY,KINGSTON,NY,12401,NY - Albany,21,20006.57,6048.85,5177.85
127575,638 - DIABETES W CC,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,AL - Dothan,32,21175.81,4678.43,4047.68
127576,638 - DIABETES W CC,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,AL - Birmingham,12,9719.16,4863.75,4203.41
127577,638 - DIABETES W CC,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,AL - Birmingham,35,17021.54,4434.57,3537.20
127578,638 - DIABETES W CC,10011,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,AL - Birmingham,14,15875.50,5176.07,3394.64


In [44]:
# Select data just for NJ data to get a baseline for Diabetes costs in New Jersery only
procedure_NJ_charges = procedure_638_charges.loc[
    hospital_data["Provider State"] == "NJ"
]

# View the DataFrame
procedure_NJ_charges.head()

,DRG Definition,Provider Id,Provider Name,Provider Street Address,Provider City,Provider State,Provider Zip Code,Hospital Referral Region Description,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
128582,638 - DIABETES W CC,310001,HACKENSACK UNIVERSITY MEDICAL CENTER,30 PROSPECT AVE,HACKENSACK,NJ,7601,NJ - Hackensack,43,45439.81,6579.39,5873.06
128583,638 - DIABETES W CC,310002,NEWARK BETH ISRAEL MEDICAL CENTER,201 LYONS AVE,NEWARK,NJ,7112,NJ - Newark,60,61179.28,9840.78,8989.73
128584,638 - DIABETES W CC,310003,PALISADES MEDICAL CTR-NY PRESBYTERIAN HEALTHCA...,7600 RIVER RD,NORTH BERGEN,NJ,7047,NJ - Hackensack,29,43669.75,7099.65,6268.68
128585,638 - DIABETES W CC,310005,HUNTERDON MEDICAL CENTER,2100 WESCOTT DRIVE,FLEMINGTON,NJ,8822,PA - Philadelphia,12,17753.91,6081.91,4423.58
128586,638 - DIABETES W CC,310006,"ST MARY'S HOSPITAL, PASSAIC",350 BOULEVARD,PASSAIC,NJ,7055,NJ - Hackensack,17,36770.88,6574.05,5843.47


In [45]:
# Using the NJ DataFrame, create a DataFrame consisting of 
# Average Medicare Payments and Provider ID columns
NJ_average_total_payments = procedure_NJ_charges[
    ["Average Total Payments", "Provider Id"]
]

# View the DataFrame
NJ_average_total_payments.head()

,Average Total Payments,Provider Id
128582,6579.39,310001
128583,9840.78,310002
128584,7099.65,310003
128585,6081.91,310005
128586,6574.05,310006


In [46]:
# The sum of NJ's average total payments
NJ_average_total_payments['Average Total Payments'].sum()

379875.32

### Step 2: Calculate the Average Total Payments by Provider State

In [71]:
# Using the procedure_638_charges DataFrame, select state and average total payments from your data.
payment_by_state = procedure_638_charges[
    ["Provider State", "Average Total Payments"]
    ]

# Aggregate the sum of payments by Provider State
total_payment_by_state = payment_by_state.groupby("Average Total Payments").sum()
#total_payment_by_state = payment_by_state.sum()

# Visualize the DataFrame
# YOUR CODE HERE
#df_total_payment_by_state = pd.DataFrame({"State" : "Provider State", "Total Payments" : total_payment_by_state})
total_payment_by_state = total_payment_by_state.reset_index()
total_payment_by_state
#df_total_payment_by_state = df_total_payment_by_state.set_index("State")
#df_total_payment_by_state.head()

,Average Total Payments,Provider State
0,3975.50,AL
1,4031.16,AL
2,4039.22,WV
3,4059.71,PA
4,4102.53,PA
...,...,...
1807,13578.12,TX
1808,14386.30,IL
1809,15566.14,CA
1810,15731.08,CA


In [72]:
total_payment_by_state.hvplot.bar()
#total_payment_by_state = total_payment_by_state.set_index('Average Total Payments')

:Bars   [index]   (Average Total Payments)

### Step 3: Plot the payment by state date in a bar chart

In [73]:
# Visualize the data using an hvPlot bar chart
# YOUR CODE HERE
#total_payment_by_state.hvplot.bar(groupby="Average Total Payments")
total_payment_by_state.hvplot.bar(
    x="Provider State",
    y=["Average Total Payments"],
    xlabel="State",
    ylabel="Total"
)

:Bars   [Provider State,Variable]   (value)

**Question:** How does the total cost of diabetes payments in New Jersey compare to that of surrounding states like NY, CT, DE, and PA?

**Answer:** # YOUR ANSWER HERE

### Step 4: Sort the cost data from low to high and then plot a bar chart


In [ ]:
# Sort the total payment by state data by Average Total Payments
# and visualize the values in a bar chart
# YOUR CODE HERE


**Question:** List the 10 states with the lowest total payments for diabetes care.

**Answer:** # YOUR ANSWER HERE

**Question:** How many states have a total diabetes payments of less than 200,000?

**Answer:** # YOUR ANSWER HERE

**Question:** What is the approximate difference in the total payments for diabetes payments between the state with the 10th highest payments and the state with the highest payments?

**Answer:** # YOUR ANSWER HERE